In [1]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

C:\Users\user\anaconda3\envs\konlpy\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


사전 훈련된 glove로 맵핑(mapping)된 단어들의 벡터 모음집(glove.txt)를 읽어 gensim의 word2vec 모델이 읽을 수 있도록 포맷(format)을 변경한다. 벡터 모음집에는 총 358,043개의 단어들에 대한 벡터들이 정의되어 있으며 100차원 벡터공간에서 표현된다.

In [2]:
input_file = "glove.txt" # https://ratsgo.github.io/embedding/downloaddata.html에서 다운로드
output_file = "tmp.txt"

In [3]:
glove2word2vec(input_file, output_file)

C:\Users\user\anaconda3\envs\konlpy\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  """Entry point for launching an IPython kernel.


(358043, 100)

In [4]:
model = KeyedVectors.load_word2vec_format(output_file, binary=False)

> 단어 "학생"과 가장 유사한 단어 출력

In [5]:
model.most_similar("학생")

[('대학생', 0.7461106777191162),
 ('교사', 0.7416083216667175),
 ('교육', 0.7392004728317261),
 ('학교', 0.7304965257644653),
 ('입학', 0.7118252515792847),
 ('수업', 0.7032736539840698),
 ('재학', 0.7020388841629028),
 ('대학', 0.6750809550285339),
 ('학년', 0.6738568544387817),
 ('청년', 0.6623227596282959)]

> 단어 "학생"을 표현하는 벡터 출력

In [6]:
model["학생"]

array([-0.442068,  0.419098,  0.265649,  0.465298,  0.436024, -0.611068,
        0.110598, -0.283077, -0.739254, -0.136494,  0.943221, -0.018234,
       -0.364238,  0.174971, -0.060601,  0.589938,  0.049765,  1.000368,
       -0.457208,  0.212553,  1.518014,  0.862085,  0.412899,  0.238667,
       -0.591947,  0.18163 ,  0.224116, -0.625323,  0.109464,  0.119204,
       -0.245809,  0.986929, -0.045883,  0.071091, -0.4182  , -0.286162,
        0.229927,  0.182633, -0.186163,  0.945371, -1.131229, -0.380229,
        1.127151,  0.054559, -0.102182, -0.798828, -0.392313, -0.164243,
        0.402235,  0.310786, -0.470877, -0.305461, -0.050434, -0.817   ,
       -0.670995, -0.690822,  0.100976, -0.36821 , -0.052327,  0.745279,
       -0.142464,  0.350418,  0.019924, -0.879077,  0.79674 , -0.243174,
       -0.291755, -2.295311, -0.880873,  0.395374,  0.141354, -0.180227,
        0.653104, -0.633723, -0.189108, -0.86881 ,  0.998197,  0.027758,
        0.135938, -0.488405,  1.113262, -0.228982, 

여기선 문장들이 담겨있는 csv파일을 읽어와 각 문장을 토크나이저 Mecab을 이용해 토큰화를 진행한다. 그 후 word to index, index to word를 생성한 후 각 문장를 단어의 index로 표현한다.

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("sample.csv", encoding="utf-8")

In [9]:
df.head()

,sentence
0,오늘 학생들의 기분은 매우 좋다
1,시험 성적이 기대된다


In [12]:
# JDK 설치 (https://webnautes.tistory.com/1394)
# 시스템에서 JAVA_HOME 설정
# Path 설정
# conda install -c conda-forge jpype1
# pip install konlpy
# whl파일 다운 및 설치 (https://www.lfd.uci.edu/~gohlke/pythonlibs/#jpype, 1.1.2 version 추천)

from konlpy.tag import Mecab
import numpy as np

m = Mecab("C:\mecab\mecab-ko-dic")
print(m.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))

['열심히', '코딩', '한', '당신', ',', '연휴', '에', '는', '여행', '을', '가', '봐요']


> 토큰화 및 word to index 생성

In [19]:
word2idx = {}
word2idx["PAD"] = 0
word2idx["UNK"] = 1

count = 2

data = list(df["sentence"])
sentences = []

for i in range(len(data)):
    data[i] = m.morphs(data[i])
    sentences.append(data[i])
    for token in data[i]:
        if token not in word2idx.keys():
            word2idx[token] = count
            count += 1
            
idx2word = {y:x for x,y in word2idx.items()}

In [16]:
word2idx

{'PAD': 0,
 'UNK': 1,
 '오늘': 2,
 '학생': 3,
 '들': 4,
 '의': 5,
 '기분': 6,
 '은': 7,
 '매우': 8,
 '좋': 9,
 '다': 10,
 '시험': 11,
 '성적': 12,
 '이': 13,
 '기대': 14,
 '된다': 15}

>index to word 생성

In [20]:
idx2word

{0: 'PAD',
 1: 'UNK',
 2: '오늘',
 3: '학생',
 4: '들',
 5: '의',
 6: '기분',
 7: '은',
 8: '매우',
 9: '좋',
 10: '다',
 11: '시험',
 12: '성적',
 13: '이',
 14: '기대',
 15: '된다'}

In [35]:
sent2idx = []
for i in range(len(sentences)):
    sample = [word2idx[j] for j in sentences[i]]
    sent2idx.append(sample)
    
print(sent2idx)

[[2, 3, 4, 5, 6, 7, 8, 9, 10], [11, 12, 13, 14, 15]]


차후 딥러닝 모델 구축 과정 중 nn.Embedding에 덮어쓸 weight를 생성한다. 단어의 index를 embedding을 위한 look up table에서 해당 단어의 임베딩 벡터가 존재해야할 위치로 사용해야 한다.

In [29]:
vocab_num = len(word2idx.keys())
weight = np.zeros((vocab_num, 100))
for i in range(2, vocab_num):
    if idx2word[i] in model.key_to_index.keys():
        weight[i] = model[idx2word[i]]

2번 index는 단어 "오늘"을 의미한다. 그렇다면 look up table에서 2번째 줄의 벡터와 pretrained glove가 생성한 "의미"를 나타내는 벡터는 동일해야 한다.

In [30]:
weight[2]

array([ 0.230271  , -0.20643599, -0.49470699, -0.006647  , -0.105034  ,
        0.40849   , -0.395722  , -0.334943  , -0.55203998,  0.296213  ,
       -0.65965998,  0.08553   ,  0.03589   , -0.077918  ,  0.35376099,
       -0.46819001, -0.061734  , -0.54861999,  0.25723901,  0.227614  ,
        1.56688404, -0.31051001, -0.22943801,  0.53551501, -0.246006  ,
        0.37057701, -0.45850301, -0.78754598,  0.17702   ,  0.35902199,
       -0.933312  , -0.37389901,  0.65083599,  0.25772801, -0.61403   ,
       -0.28142601,  0.139157  ,  0.85244697,  0.08039   ,  0.346275  ,
        0.14231899,  0.47718999,  0.23139501,  0.019545  ,  0.353598  ,
       -0.20912801, -0.42031601, -0.69834697,  1.10519803,  0.51489198,
        0.19998901,  0.172005  ,  1.10531998, -0.146338  , -0.15420499,
        0.036494  ,  0.16675501, -0.329153  , -0.039567  , -0.098935  ,
        0.072235  ,  0.316726  , -0.57284302,  0.67462403,  0.108507  ,
        0.26225099,  0.23825499, -1.44127202, -0.34044299,  0.04

In [31]:
model["오늘"]

array([ 0.230271, -0.206436, -0.494707, -0.006647, -0.105034,  0.40849 ,
       -0.395722, -0.334943, -0.55204 ,  0.296213, -0.65966 ,  0.08553 ,
        0.03589 , -0.077918,  0.353761, -0.46819 , -0.061734, -0.54862 ,
        0.257239,  0.227614,  1.566884, -0.31051 , -0.229438,  0.535515,
       -0.246006,  0.370577, -0.458503, -0.787546,  0.17702 ,  0.359022,
       -0.933312, -0.373899,  0.650836,  0.257728, -0.61403 , -0.281426,
        0.139157,  0.852447,  0.08039 ,  0.346275,  0.142319,  0.47719 ,
        0.231395,  0.019545,  0.353598, -0.209128, -0.420316, -0.698347,
        1.105198,  0.514892,  0.199989,  0.172005,  1.10532 , -0.146338,
       -0.154205,  0.036494,  0.166755, -0.329153, -0.039567, -0.098935,
        0.072235,  0.316726, -0.572843,  0.674624,  0.108507,  0.262251,
        0.238255, -1.441272, -0.340443,  0.04393 ,  0.093036, -0.837977,
        0.463689,  0.468401, -0.078953,  0.81616 ,  0.614467,  0.252222,
        0.557586, -0.472108,  0.337196,  0.33712 , 